In [4]:
from sympy import *
import numpy as np

In [10]:
#열역학 변수들
[dU,T,dS,P,dV,mu,dN]=[Symbol('dU'),Symbol('T'),Symbol('dS'),Symbol('P'),Symbol('dV'),Symbol('mu'),Symbol('dN')]

In [65]:
nmat=np.array([[T,dS,-P,dV,mu,dN,-1,dU]])# 열역학 제 1법칙을 표현함; TdS-PdV+mudN-dU=0

nmat=np.vstack((nmat,nmat,nmat,nmat)) #똑같은 4개의 세트를 쌓은 다음에 3개의 다른 관계식을 만들 것이다. 
nmat

array([[T, dS, -P, dV, mu, dN, -1, dU],
       [T, dS, -P, dV, mu, dN, -1, dU],
       [T, dS, -P, dV, mu, dN, -1, dU],
       [T, dS, -P, dV, mu, dN, -1, dU]], dtype=object)

In [66]:
for i in range(3):
    denom=-1*nmat[0,2*i]  # T, -P, mu
    for j in range(4):
        nmat[i+1,2*j]/=denom
        
nmat  #열역학 1법칙과 파생된 관계식들

array([[T, dS, -P, dV, mu, dN, -1, dU],
       [-1, dS, P/T, dV, -mu/T, dN, 1/T, dU],
       [T/P, dS, -1, dV, mu/P, dN, -1/P, dU],
       [-T/mu, dS, P/mu, dV, -1, dN, 1/mu, dU]], dtype=object)

In [67]:
'''
이항하기
'''

for i in range(3):
    tmp1,tmp2=nmat[i+1,2*i],nmat[i+1,2*i+1]
    nmat[i+1,2*i],nmat[i+1,2*i+1]=nmat[i+1,6],nmat[i+1,7]
    nmat[i+1,6],nmat[i+1,7]=tmp1,tmp2
    
nmat

array([[T, dS, -P, dV, mu, dN, -1, dU],
       [1/T, dU, P/T, dV, -mu/T, dN, -1, dS],
       [T/P, dS, -1/P, dU, mu/P, dN, -1, dV],
       [-T/mu, dS, P/mu, dV, 1/mu, dU, -1, dN]], dtype=object)

In [68]:
Maxwell=nmat[:,0:6] # 열 자르기 : 0열부터 5열까지 살리고 6열부터 자르기
Maxwell

array([[T, dS, -P, dV, mu, dN],
       [1/T, dU, P/T, dV, -mu/T, dN],
       [T/P, dS, -1/P, dU, mu/P, dN],
       [-T/mu, dS, P/mu, dV, 1/mu, dU]], dtype=object)

In [73]:
Max=Maxwell.reshape(4,3,2) # rank 3짜리 텐서로 만들면 다루기 쉬워진다. 
Max

array([[[T, dS],
        [-P, dV],
        [mu, dN]],

       [[1/T, dU],
        [P/T, dV],
        [-mu/T, dN]],

       [[T/P, dS],
        [-1/P, dU],
        [mu/P, dN]],

       [[-T/mu, dS],
        [P/mu, dV],
        [1/mu, dU]]], dtype=object)

In [76]:
Max[0]

array([[T, dS],
       [-P, dV],
       [mu, dN]], dtype=object)

In [104]:
g = np.empty((4,3,4),dtype="U6")  #3행 4열짜리 행렬이 4개 들어있는 빈 텐서를 짜 놓는다 여기에 맥스웰 관계식들이 들어갈 것이다. 

In [105]:
# 어떻게 들어갈 건지 해본 것 

for k in range(3):
    g[0][k,0],g[0][k,1],g[0][k,2],g[0][k,3]=Max[0][k-1,1],Max[0][k,0],Max[0][k,1],Max[0][k-1,0]
    
g[0]

array([['dN', 'T', 'dS', 'mu'],
       ['dS', '-P', 'dV', 'T'],
       ['dV', 'mu', 'dN', '-P']], dtype='<U6')

In [106]:
for i in range(4):
    for j in range(3):
            g[i][j,0],g[i][j,1],g[i][j,2],g[i][j,3]=Max[i][j-1,1],Max[i][j,0],Max[i][j,1],Max[i][j-1,0]

g  

array([[['dN', 'T', 'dS', 'mu'],
        ['dS', '-P', 'dV', 'T'],
        ['dV', 'mu', 'dN', '-P']],

       [['dN', '1/T', 'dU', '-mu/T'],
        ['dU', 'P/T', 'dV', '1/T'],
        ['dV', '-mu/T', 'dN', 'P/T']],

       [['dN', 'T/P', 'dS', 'mu/P'],
        ['dS', '-1/P', 'dU', 'T/P'],
        ['dU', 'mu/P', 'dN', '-1/P']],

       [['dU', '-T/mu', 'dS', '1/mu'],
        ['dS', 'P/mu', 'dV', '-T/mu'],
        ['dV', '1/mu', 'dU', 'P/mu']]], dtype='<U6')

여기서 'dN'은 $ \partial_{N}$ 으로 이해하면 된다. 따라서 ['dN' , 'T' , 'dS' , 'mu' ]는 $\partial_{N}\, T=\partial_{S}\, \mu$ 이렇게 보면 된다

In [107]:
g=g.reshape((12,4))  #전부 합쳐서 데이터프레임속에 넣어두지

In [108]:
g

array([['dN', 'T', 'dS', 'mu'],
       ['dS', '-P', 'dV', 'T'],
       ['dV', 'mu', 'dN', '-P'],
       ['dN', '1/T', 'dU', '-mu/T'],
       ['dU', 'P/T', 'dV', '1/T'],
       ['dV', '-mu/T', 'dN', 'P/T'],
       ['dN', 'T/P', 'dS', 'mu/P'],
       ['dS', '-1/P', 'dU', 'T/P'],
       ['dU', 'mu/P', 'dN', '-1/P'],
       ['dU', '-T/mu', 'dS', '1/mu'],
       ['dS', 'P/mu', 'dV', '-T/mu'],
       ['dV', '1/mu', 'dU', 'P/mu']], dtype='<U6')

In [109]:
g0=g[:,0:2]  #좌변
g1=g[:,2:4]  #우변

In [110]:
g0

array([['dN', 'T'],
       ['dS', '-P'],
       ['dV', 'mu'],
       ['dN', '1/T'],
       ['dU', 'P/T'],
       ['dV', '-mu/T'],
       ['dN', 'T/P'],
       ['dS', '-1/P'],
       ['dU', 'mu/P'],
       ['dU', '-T/mu'],
       ['dS', 'P/mu'],
       ['dV', '1/mu']], dtype='<U6')

In [111]:
g1

array([['dS', 'mu'],
       ['dV', 'T'],
       ['dN', '-P'],
       ['dU', '-mu/T'],
       ['dV', '1/T'],
       ['dN', 'P/T'],
       ['dS', 'mu/P'],
       ['dU', 'T/P'],
       ['dN', '-1/P'],
       ['dS', '1/mu'],
       ['dV', '-T/mu'],
       ['dU', 'P/mu']], dtype='<U6')

In [7]:
import pandas as pd
from pandas import DataFrame

In [130]:
equiv=np.array([['='],['='],['='],['='],['='],['='],['='],['='],['='],['='],['='],['=']]) #보기 좋게 하기위해 등호 데이터프레임도 넣어준다.
df1=DataFrame(g0)
df2=DataFrame(g1)
EQ=DataFrame(equiv)

In [136]:
df=pd.concat([df1,EQ,df2],axis=1)
df

,0,1,0,0,1
0,dN,T,=,dS,mu
1,dS,-P,=,dV,T
2,dV,mu,=,dN,-P
3,dN,1/T,=,dU,-mu/T
4,dU,P/T,=,dV,1/T
5,dV,-mu/T,=,dN,P/T
6,dN,T/P,=,dS,mu/P
7,dS,-1/P,=,dU,T/P
8,dU,mu/P,=,dN,-1/P
9,dU,-T/mu,=,dS,1/mu


자유에너지의 경우도 한번 보자!

In [151]:
# dF=-SdT-PdV+mudN
[dF,S,dT,P,dV,mu,dN]=[Symbol('dF'),Symbol('S'),Symbol('dT'),Symbol('P'),Symbol('dV'),Symbol('mu'),Symbol('dN')]
nmat=np.array([[-S,dT,-P,dV,mu,dN,-1,dF]])# 열역학 제 1법칙을 표현함; TdS-PdV+mudN-dU=0

nmat=np.vstack((nmat,nmat,nmat,nmat)) #똑같은 4개의 세트를 쌓은 다음에 3개의 다른 관계식을 만들 것이다. 
    
for i in range(3):
    denom=-1*nmat[0,2*i]  # T, -P, mu
    for j in range(4):
        nmat[i+1,2*j]/=denom
    
    '''
    이항하기
    '''

for i in range(3):
    tmp1,tmp2=nmat[i+1,2*i],nmat[i+1,2*i+1]
    nmat[i+1,2*i],nmat[i+1,2*i+1]=nmat[i+1,6],nmat[i+1,7]
    nmat[i+1,6],nmat[i+1,7]=tmp1,tmp2
        
Maxwell=nmat[:,0:6] #열 자르기
    
Max=Maxwell.reshape(4,3,2) # rank 3짜리 텐서로 만들면 다루기 쉬워진다.
    
F = np.empty((4,3,4),dtype="U6")  #3행 4열짜리 행렬이 4개 들어있는 빈 텐서를 짜 놓는다 여기에 맥스웰 관계식들이 들어갈 것이다. 

for i in range(4):
    for j in range(3):
            F[i][j,0],F[i][j,1],F[i][j,2],F[i][j,3]=Max[i][j-1,1],Max[i][j,0],Max[i][j,1],Max[i][j-1,0]
            
F=F.reshape((12,4))  #전부 합쳐서 데이터프레임속에 넣어두지

F0=F[:,0:2]  #좌변
F1=F[:,2:4]  #우변

equiv=np.array([['='],['='],['='],['='],['='],['='],['='],['='],['='],['='],['='],['=']]) #보기 좋게 하기위해 등호 데이터프레임도 넣어준다.
df1=DataFrame(F0)
df2=DataFrame(F1)
EQ=DataFrame(equiv)

df=pd.concat([df1,EQ,df2],axis=1)

In [152]:
df

,0,1,0,0,1
0,dN,-S,=,dT,mu
1,dT,-P,=,dV,-S
2,dV,mu,=,dN,-P
3,dN,-1/S,=,dF,mu/S
4,dF,-P/S,=,dV,-1/S
5,dV,mu/S,=,dN,-P/S
6,dN,-S/P,=,dT,mu/P
7,dT,-1/P,=,dF,-S/P
8,dF,mu/P,=,dN,-1/P
9,dF,S/mu,=,dT,1/mu


총 12개의 관계식을 얻을 수 있다. 함수로도 만들 수 있다.

In [2]:
def MaxRelation(x1,x2,x3,x4,x5,x6,x7,x8):
    mat=np.array([[x1,x2,x3,x4,x5,x6,x7,x8]])
    nmat=np.vstack((mat,mat,mat,mat))
    
    for i in range(3):
        denom=-1*nmat[0,2*i]  # T, -P, mu
        for j in range(4):
            nmat[i+1,2*j]/=denom
            
    
    for i in range(3):
        tmp1,tmp2=nmat[i+1,2*i],nmat[i+1,2*i+1]
        nmat[i+1,2*i],nmat[i+1,2*i+1]=nmat[i+1,6],nmat[i+1,7]
        nmat[i+1,6],nmat[i+1,7]=tmp1,tmp2
        
    Maxwell=nmat[:,0:6] #열 자르기
    Max=Maxwell.reshape(4,3,2)
    
    F = np.empty((4,3,4),dtype="U6")  #3행 4열짜리 행렬이 4개 들어있는 빈 텐서를 짜 놓는다 여기에 맥스웰 관계식들이 들어갈 것이다. 

    for i in range(4):
        for j in range(3):
            F[i][j,0],F[i][j,1],F[i][j,2],F[i][j,3]=Max[i][j-1,1],Max[i][j,0],Max[i][j,1],Max[i][j-1,0]
            
    F=F.reshape((12,4))  #전부 합쳐서 데이터프레임속에 넣어두지

    F0=F[:,0:2]  #좌변
    F1=F[:,2:4]  #우변

    equiv=np.array([['='],['='],['='],['='],['='],['='],['='],['='],['='],['='],['='],['=']]) #보기 좋게 하기위해 등호 데이터프레임도 넣어준다.
    df1=DataFrame(F0)
    df2=DataFrame(F1)
    EQ=DataFrame(equiv)

    df=pd.concat([df1,EQ,df2],axis=1)
    
    return df

엔탈피를 한번 보자 $$dH=TdS+\mu dN+VdP, \quad TdS+\mu dN+VdP-dH=0$$

MaxRelation 함수에 들어갈 변수는 {T,dS,mu,dN,V,dP,-1,dH} 이고 함수에 넣기 전에 Symbolic하게 만들어서 넣는것을 잊으면 안된다.

In [5]:
T,dS,mu,dN,V,dP,dH = Symbol('T'),Symbol('dS'),Symbol('mu'),Symbol('dN'),Symbol('V'),Symbol('dP'),Symbol('dH')

In [8]:
MaxRelation(T,dS,mu,dN,V,dP,-1,dH)

,0,1,0,0,1
0,dP,T,=,dS,V
1,dS,mu,=,dN,T
2,dN,V,=,dP,mu
3,dP,1/T,=,dH,-V/T
4,dH,-mu/T,=,dN,1/T
5,dN,-V/T,=,dP,-mu/T
6,dP,-T/mu,=,dS,-V/mu
7,dS,1/mu,=,dH,-T/mu
8,dH,-V/mu,=,dP,1/mu
9,dH,-T/V,=,dS,1/V


Gibbs free energy: $dG=-SdT+VdP+\mu dN, \quad -SdT+VdP+\mu dN-dG=0$

In [13]:
S,dT,V,dP,mu,dN,dG = Symbol('S'),Symbol('dT'),Symbol('V'),Symbol('dP'),Symbol('mu'),Symbol('dN'),Symbol('dG')

In [14]:
MaxRelation(-S,dT,V,dP,mu,dN,-1,dG)

,0,1,0,0,1
0,dN,-S,=,dT,mu
1,dT,V,=,dP,-S
2,dP,mu,=,dN,V
3,dN,-1/S,=,dG,mu/S
4,dG,V/S,=,dP,-1/S
5,dP,mu/S,=,dN,V/S
6,dN,S/V,=,dT,-mu/V
7,dT,1/V,=,dG,S/V
8,dG,-mu/V,=,dN,1/V
9,dG,S/mu,=,dT,1/mu


이렇게 해서 48개의 관계식을 모두 구했다!